# Assignment 1: Preprocessing and Text Classification

Student Name: Chi Zhang

Student ID: 1067750

# General Info

<b>Due date</b>: Monday, 27 March 2023 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python" page on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3.8</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. In regards to the use of artificial intelligence tools in the context of academic integrity, please see the university's statement <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be working with a collection of tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [1]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])
    
assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its going down🙏🙌  @BishopJakes
Country = gb 	Tweet = @EllexxxPharrell wow love the pic babe xx
Country = us 	Tweet = You 

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation (based on frequency). The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English letters in the alphabet (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet (represented using a python dictionary). For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [2]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    # Your answer BEGINS HERE
    ###
    x_result = []
    y_result = []

    # Tokenisation of each sentence
    for i in range(len(data)):
        BOW = {}
        token_result = tt.tokenize(data[i])

        # Turn all token(s) in the list to lowercase
        lower_token = (list(map(str.lower, token_result)))

        # Remove token without alphabet
        for token in lower_token:
            if (re.search('[a-z]', token) != None):
                # Drop the token belongs to stopwords
                if token not in stopwords:
                    BOW[token] = BOW.get(token, 0) + 1

        # Check the BOW is empty or not
        if (len(BOW) > 0):
            x_result.append(BOW)
            y_result.append(labels[i])
    return x_result, y_result

    ###
    # Your answer ENDS HERE
    ###

x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
for i in range(10):
    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
Country = us 	Tweet = {'@addictd2success': 1, 'thx': 1, 'u': 1, 'following': 1}
Country = us 	Tweet = {"let's": 1, 'say': 1, 'ever': 1, 'switch': 1, 'teams': 1, 'khalesi': 1, 'would': 1, 'top': 1, 'list': 1, '#girlcrush': 1}
Country = ph 	Tweet = {'taemin': 1, 'jonghyun': 1, 'birits': 1, 'make': 1, 'go': 1, 'http://t.co/le8z3dntla': 1}
Country = id 	Tweet = {'depart.senior': 1, 'rapat': 1, 'perdana': 1, 'nyayu': 1, 'anita': 1, 'others': 1, 'ruang': 1, 'aescullap': 1, 'fk': 1, 'unsri': 1, 'madang': 1, 'https://t.co/swrallnkrq': 1}
Country = ph 	Tweet = {'done': 1, 'internship': 1, 'pretty': 1, 'little': 1, 'lady': 1, 'metropolitan': 1, 'medical': 1, 'center': 1, 'w': 1, 'others': 1, 'pic': 1, 'http://t.co/1qh61r1t5r': 1}
Country = gb 	Tweet = {'wow': 1, "boruc's": 1, 'clanger': 1, 'haha': 1, 'sunday': 1, 'league': 1, 'stuff': 1, 'giroud': 1, 'believe': 1, 'luck': 1, '#clown': 1}
Countr

[nltk_data] Error loading stopwords: <urlopen error [Errno 60]
[nltk_data]     Operation timed out>


**For your testing**:

In [3]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [4]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecords", the algorithm should produce: \["#", "new", "records"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import wordnet
nltk.download('words')
nltk.download('wordnet')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    result_dict = {}
    for hashtag in hashtags:
        token_list = []
        next_start = 0
        while next_start < len(hashtag):
            store_token = hashtag[next_start]
            for i in range(next_start, len(hashtag)):
                current_token = hashtag[next_start:(i+1)]
                if lemmatize(current_token) in words:
                    store_token = current_token
            next_start += len(store_token)
            token_list.append(store_token)
        result_dict[hashtag] = token_list
    return result_dict
    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

[nltk_data] Error loading words: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>
[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>


#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'coca', 'm']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '9', '0', 's']
#wearesocial ['#', 'weares', 'o', 'c', 'i', 'al']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [6]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
def tokenize_hashtags_rev(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    result_dict = {}
    for hashtag in hashtags:
        token_list = []
        current_token_len = len(hashtag)
        while current_token_len > 0:
            store_token = hashtag[current_token_len-1]
            for i in range(current_token_len, 0, -1):
                current_token = hashtag[(i-1):current_token_len]
                if lemmatize(current_token) in words:
                    store_token = current_token
            current_token_len -= len(store_token)
            token_list.insert(0, store_token)
        result_dict[hashtag] = token_list
    return result_dict
    ###
    # Your answer ENDS HERE
    ###

    
#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'gan']
#veganfood ['#', 'v', 'e', 'gan', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'c', 'o', 'cam']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'we', 'are', '9', '0', 's']
#wearesocial ['#', 'we', 'are', 'social']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'a', 'mas']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [8]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hastag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hellow</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [9]:
from nltk.corpus import brown
import math
from collections import Counter

#words from brown corpus
brown_words = brown.words()

###
# Your answer BEGINS HERE
###
# Turn all words into lowercase
brown_words_low = (list(map(str.lower, brown_words)))

# Store the total number of words from the corpus
M = len(brown_words_low)

# Store the count of token in Counter and compute the count of Vocabulary
uni_count = Counter()
for i in brown_words_low:
    uni_count[i] += 1
V = len(uni_count)

# Compute the LogProb of Hashtags from MaxMatch and reversed MaxMatch based on Unigram with add-1 smoothing
diff_hash_index = 0
for hashtag in hashtags:
    mm_token = tokenized_hashtags[hashtag]
    rmm_token = tokenized_hashtags_rev[hashtag]
    if mm_token != rmm_token:
        diff_hash_index += 1
        mm_logprob = round(sum([math.log((uni_count[token]+1)/(M+V)) for token in mm_token]), 1)
        rmm_logprob = round(sum([math.log((uni_count[token]+1)/(M+V)) for token in rmm_token]), 1)
        print(str(diff_hash_index) + ". " + hashtag)
        print("MaxMatch = " + str(mm_token) + "; LogProb = " + str(mm_logprob))
        print("Reverved MaxMatch = " + str(mm_token) + "; LogProb = " + str(rmm_logprob) + "\n")
###
# Your answer ENDS HERE
###

1. #goingout
MaxMatch = ['#', 'going', 'out']; LogProb = -28.4
Reverved MaxMatch = ['#', 'going', 'out']; LogProb = -38.5

2. #uniexamonasaturday
MaxMatch = ['#', 'unie', 'x', 'am', 'ona', 'saturday']; LogProb = -71.6
Reverved MaxMatch = ['#', 'unie', 'x', 'am', 'ona', 'saturday']; LogProb = -80.0

3. #usydoweek
MaxMatch = ['#', 'us', 'y', 'dow', 'e', 'e', 'k']; LogProb = -78.3
Reverved MaxMatch = ['#', 'us', 'y', 'dow', 'e', 'e', 'k']; LogProb = -48.9

4. #imsobored
MaxMatch = ['#', 'i', 'ms', 'o', 'bored']; LogProb = -54.4
Reverved MaxMatch = ['#', 'i', 'ms', 'o', 'bored']; LogProb = -48.4

5. #ididntchoosethestudentlifeitchoseme
MaxMatch = ['#', 'id', 'id', 'n', 'tch', 'o', 'ose', 'the', 'student', 'life', 'itch', 'ose', 'me']; LogProb = -143.7
Reverved MaxMatch = ['#', 'id', 'id', 'n', 'tch', 'o', 'ose', 'the', 'student', 'life', 'itch', 'ose', 'me']; LogProb = -95.9

6. #laundryservice
MaxMatch = ['#', 'laundrys', 'er', 'vice']; LogProb = -52.3
Reverved MaxMatch = ['#', 'laundrys'

# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this (but if you don't want to use sklearn that's fine).

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

###
# Your answer BEGINS HERE
###
# Split the dataset into train, development, test partitions
x_train, x_l, y_train, y_l = train_test_split(x_processed, y_processed, test_size=0.3, random_state=42, stratify=y_processed)
x_dev, x_test, y_dev, y_test = train_test_split(x_l, y_l, test_size = 0.5, random_state = 42, stratify=y_l)

# Vectorize the splitted dataset
vectorizer = DictVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_dev = vectorizer.transform(x_dev)
x_test = vectorizer.transform(x_test)
###
# Your answer ENDS HERE
###

### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import warnings

###
# Your answer BEGINS HERE
###
warnings.filterwarnings("ignore")

''' Function developed based on Tutorial 03-classification.ipynb for hyper-parameters tuning with cross validation'''
def do_multiple_10foldcrossvalidation(clfs, data, classifications):
    best_accuracy = 0
    for clf in clfs:
        predictions = model_selection.cross_val_predict(clf, data, classifications, cv=10)
        current_accuracy = accuracy_score(classifications, predictions)
        print(clf)
        print("accuracy = ", current_accuracy)

        # Record the best accuracy and the best model
        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_model = clf
    print()
    return best_accuracy, best_model

# Base on sklearn.naive_bayes.MultinomialNB documentation, tune the hyper-parameter alpha (a) with default alpha = 1
a_tune = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
mbs = [MultinomialNB(alpha=a) for a in a_tune]
best_accuracy_mb, best_model_mb = do_multiple_10foldcrossvalidation(mbs, x_dev, y_dev)
print("The best Hyper-parameter alpha =", best_model_mb.alpha, "in MultinomialNB to achieve accuracy = ", best_accuracy_mb)
print("\n")

# Base on sklearn.linear_model.LogisticRegression documentation, tune the hyper-parameter C with default 'l2' penalty
c_tune = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
lrcs = [LogisticRegression(C=c) for c in c_tune]
best_accuracy_lr, best_model_lr = do_multiple_10foldcrossvalidation(lrcs, x_dev, y_dev)
print("The best Hyper-parameter C =", best_model_lr.C, "in LogisticRegression to achieve accuracy = ", best_accuracy_lr)
###
# Your answer ENDS HERE
###

MultinomialNB(alpha=0.001)
accuracy =  0.16312056737588654
MultinomialNB(alpha=0.01)
accuracy =  0.16312056737588654
MultinomialNB(alpha=0.1)
accuracy =  0.14184397163120568
MultinomialNB(alpha=1)
accuracy =  0.1276595744680851
MultinomialNB(alpha=10)
accuracy =  0.1347517730496454
MultinomialNB(alpha=100)
accuracy =  0.05673758865248227
MultinomialNB(alpha=1000)
accuracy =  0.05673758865248227

The best Hyper-parameter alpha = 0.001 in MultinomialNB to achieve accuracy =  0.16312056737588654


LogisticRegression(C=0.001)
accuracy =  0.0851063829787234
LogisticRegression(C=0.01)
accuracy =  0.0851063829787234
LogisticRegression(C=0.1)
accuracy =  0.15602836879432624
LogisticRegression(C=1)
accuracy =  0.1702127659574468
LogisticRegression(C=10)
accuracy =  0.15602836879432624
LogisticRegression(C=100)
accuracy =  0.14893617021276595
LogisticRegression(C=1000)
accuracy =  0.1276595744680851

The best Hyper-parameter C = 1 in LogisticRegression to achieve accuracy =  0.1702127659574468


### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [12]:
###
# Your answer BEGINS HERE
###
# Import f1-score for computing macro-averaged F-score
from sklearn.metrics import f1_score

# Train the MultinomialNB with alpha = 0.01 and test it with x_test, y_test
best_mb = MultinomialNB(alpha = best_model_mb.alpha).fit(x_train, y_train)
predictions = best_mb.predict(x_test)

# Compute and print the accuracy and macro-averaged F-score of the MultinomialNB model
acc = accuracy_score(y_test, predictions)
maf = f1_score(y_test, predictions, average = 'macro')
print("Naive Bayes with alpha = 0.01 has accuracy =", acc, ', macro-averaged F-score =', maf)
print()

# Train the LogisticRegression with C = 1 and test it with x_test, y_test
best_lr = LogisticRegression(C = best_model_lr.C).fit(x_train, y_train)
predictions = best_lr.predict(x_test)

# Compute and print the accuracy and macro-averaged F-score of the LogisticRegression model
acc = accuracy_score(y_test, predictions)
maf = f1_score(y_test, predictions, average = 'macro')
print("Logistic Regression with C = 1 has accuracy =", acc, ', macro-averaged F-score =', maf)
###
# Your answer ENDS HERE
###

Naive Bayes with alpha = 0.01 has accuracy = 0.28169014084507044 , macro-averaged F-score = 0.27148547076390883

Logistic Regression with C = 1 has accuracy = 0.2605633802816901 , macro-averaged F-score = 0.24441374917278252


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [13]:
###
# Your answer BEGINS HERE
###
# Record the classes name and features name
country_class = list(best_lr.classes_)
features_name = list(vectorizer.get_feature_names_out())

# Record the coefficient value from LogisticRegression and MultinomialNB with sklearn attributes
lr_coef = best_lr.coef_
mb_coef = best_mb.feature_log_prob_

# Extract the top-20 features
top_20 = 20

# Extract top-20 features and their coefficient values in each country (LogisticRegression)
print("Classifier = Logistic Regression", '\n')
for i in range(len(country_class)):
    print("Country =", country_class[i])
    feature_dict = {}
    for f in range(len(features_name)):
        feature_dict[features_name[f]] = lr_coef[i][f]
    sorted_feature = dict(sorted(feature_dict.items(), key=lambda x: x[1], reverse=True))
    top_20_features = dict(list(sorted_feature.items())[:top_20])
    for top_feature in top_20_features:
        print(top_feature, f"({round(top_20_features[top_feature], 3)})", end = " ")
    print("\n")

# Extract top-20 features and their coefficient values in each country (MultinomialNB)
print("Classifier = Naive Bayes", "\n")   
for i in range(len(country_class)):
    print("Country =", country_class[i])
    feature_dict = {}
    for f in range(len(features_name)):
        feature_dict[features_name[f]] = mb_coef[i][f]
    sorted_feature = dict(sorted(feature_dict.items(), key=lambda x: x[1], reverse=True))
    top_20_features = dict(list(sorted_feature.items())[:top_20])
    for top_feature in top_20_features:
        print(top_feature, f"({round(top_20_features[top_feature], 1)})", end = " ")
    print("\n")
###
# Your answer ENDS HERE
###

Classifier = Logistic Regression 

Country = au
australia (1.134) melbourne (1.119) little (1.008) #melbourne (0.922) @micksunnyg (0.821) australian (0.723) beach (0.672) green (0.66) https://t.co/7rcjjptvl7 (0.636) summerpoyi's (0.636) @dasheryoung (0.625) lachie (0.625) great (0.624) away (0.622) one (0.62) @jackgilinsky (0.595) @whennboys (0.595) literally (0.591) please (0.59) someone (0.576) 

Country = ca
happen (0.898) thing (0.863) hate (0.81) gonna (0.788) learning (0.771) bed (0.765) right (0.738) healthy (0.704) really (0.697) @aliclarke (0.678) think (0.66) awesome (0.654) first (0.648) carlyle (0.645) manor (0.645) school (0.624) @cheetahbiatch (0.615) xoxo (0.615) found (0.609) got (0.606) 

Country = de
night (0.818) posted (0.8) done (0.729) https://t.co/df7ficsci3 (0.719) roseninsel (0.719) #truegrip (0.667) #utopia (0.667) gauting (0.667) https://t.co/brkwmsvzrb (0.667) hyde (0.667) jessica (0.667) bad (0.661) could (0.66) enough (0.641) painting (0.615) @kelsxmclaugh